In [1]:
import random
from configs.config import PROJECT_DIR
from src.dataset.utils import *
from src.dataset.goemotion import GoEmotionDataset

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/haiduong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haiduong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/haiduong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haiduong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load dataset

In [2]:
ds = GoEmotionDataset(PROJECT_DIR + "/data/original")

2025-02-26 21:32:08.764 | INFO     | src.dataset.goemotion:_load_labels_dict:95 - Loading labels successfully. There are 28 in total.
2025-02-26 21:32:08.956 | INFO     | src.dataset.goemotion:_load_raw_dataset:114 - Load train dataset successfully
2025-02-26 21:32:08.963 | INFO     | src.dataset.goemotion:_load_raw_dataset:114 - Load test dataset successfully
2025-02-26 21:32:08.971 | INFO     | src.dataset.goemotion:_load_raw_dataset:114 - Load dev dataset successfully


# Visualize random sample

In [3]:
raw_ds = ds.get_raw_dataset("train")

In [4]:
labels = ds._label_dict

In [5]:
labels

{0: 'admiration',
 1: 'amusement',
 2: 'anger',
 3: 'annoyance',
 4: 'approval',
 5: 'caring',
 6: 'confusion',
 7: 'curiosity',
 8: 'desire',
 9: 'disappointment',
 10: 'disapproval',
 11: 'disgust',
 12: 'embarrassment',
 13: 'excitement',
 14: 'fear',
 15: 'gratitude',
 16: 'grief',
 17: 'joy',
 18: 'love',
 19: 'nervousness',
 20: 'optimism',
 21: 'pride',
 22: 'realization',
 23: 'relief',
 24: 'remorse',
 25: 'sadness',
 26: 'surprise',
 27: 'neutral'}

In [6]:
visualize(raw_ds[random.randint(0, len(raw_ds))], labels)

Text:  Happy birthday! Drinking one for you tonight. When you get up tomorrow you can take the first step to making sure next years birthday will be better!
Label:  excitement 


# Collect key metrics
- Number of samples
- Number of classes
- Number of samples per class
- Number of words per sample
- Frequency distribution of words
- Distribution of sample length

**NOTE**: The dataset is imbalanced significantly. Especially class 'neutral' with around 14.000 samples 

# Model selection 
By following this [guide](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from google, here are the list of experiment that i will perform
- MLP
- sepCNN
- RNN
- StackedRNN
- Transformer

# Build model

In [7]:
vocab = ds.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

In [8]:
print(ds.get_vocabulary_length())

20000


In [10]:
EMBEDDING_DIM = 100

In [11]:
embedding_index = get_embedding_index(PROJECT_DIR + "/models/glove.6B/glove.6B.100d.txt")
embedding_matrix = prepare_embedding_matrix(embedding_dim=100, vocabulary_length=len(vocab), word_index=word_index, embedding_index=embedding_index)

Converted 15592 words (4408 misses)


In [12]:
print(embedding_matrix.shape)

(20000, 100)


# Build model

In [13]:
from keras.layers import Embedding

# Transfer learning
embedding_layer = Embedding(
    len(vocab),
    EMBEDDING_DIM,
    trainable=False, # We don't want to update this
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [14]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 3, padding="same", activation="relu")(embedded_sequences)
x = layers.MaxPooling1D()(x)
x = layers.Conv1D(128, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling1D(3)(x)
x = layers.Conv1D(128, 3, padding="same", activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(labels), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, None, 128)      │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 128)      │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 128)      │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │         3,612 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,157,212 (8.23 MB)

 Trainable params: 157,212 (614.11 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

In [15]:
train_ds = ds.get_preprocessed_data("train")
val_ds = ds.get_preprocessed_data("dev")
test_ds = ds.get_preprocessed_data("test")

In [18]:
model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["acc"]
)
model.fit(train_ds["features"], train_ds["labels"], batch_size=128, epochs=20, validation_data=(val_ds["features"], val_ds["labels"]))

Epoch 1/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - acc: 0.1101 - loss: 45621046149120.0000 - val_acc: 0.2934 - val_loss: 28839539376128.0000
Epoch 2/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - acc: 0.2912 - loss: 24862001201152.0000 - val_acc: 0.2934 - val_loss: 13044451115008.0000
Epoch 3/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - acc: 0.2975 - loss: 9935192588288.0000 - val_acc: 0.2934 - val_loss: 584902574080.0000
Epoch 4/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - acc: 0.2967 - loss: 77086146560.0000 - val_acc: 0.2934 - val_loss: 5756114432.0000
Epoch 5/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - acc: 0.2779 - loss: 6602304512.0000 - val_acc: 0.2934 - val_loss: 4029027840.0000
Epoch 6/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - acc: 0.2550 - loss: 4462361600.0000 - val_acc: 0.0899 - val_loss: 2547506432.0000
Epoch 7/20
231/340 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.1654 - loss: 2606510336.0000

KeyboardInterrupt: 